In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

import findspark
findspark.init()

# Create Spark session
spark = SparkSession.builder.master('local').appName("NullCountExample").getOrCreate()

# Sample data
data = [
    (1, "Alice", None),
    (2, None, "HR"),
    (3, "Charlie", "IT"),
    (None, "David", "Finance"),
    (5, None, None)
]

columns = ["id", "name", "department"]

# Create DataFrame
df = spark.createDataFrame(data, columns)

# Show the DataFrame
df.show()


+----+-------+----------+
|  id|   name|department|
+----+-------+----------+
|   1|  Alice|      NULL|
|   2|   NULL|        HR|
|   3|Charlie|        IT|
|NULL|  David|   Finance|
|   5|   NULL|      NULL|
+----+-------+----------+



In [ ]:
#let first count each column count
df.select([count(c) for c in df.columns]).show() #another example

+---------+-----------+-----------------+
|count(id)|count(name)|count(department)|
+---------+-----------+-----------------+
|        4|          3|                3|
+---------+-----------+-----------------+



In [10]:
#answer
null_counts = df.select(
    [ sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df.columns]
    )

null_counts.show()


+---+----+----------+
| id|name|department|
+---+----+----------+
|  1|   2|         2|
+---+----+----------+



In [ ]:
#this is how it is working
k=[sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df.columns]
print(k)

[Column<'sum(CASE WHEN (id IS NULL) THEN 1 ELSE 0 END) AS id'>, Column<'sum(CASE WHEN (name IS NULL) THEN 1 ELSE 0 END) AS name'>, Column<'sum(CASE WHEN (department IS NULL) THEN 1 ELSE 0 END) AS department'>]


In [16]:
null_counts.createOrReplaceTempView('nullcount')

In [17]:
%%sql
select * from nullcount


+---+----+----------+
| id|name|department|
+---+----+----------+
|  1|   2|         2|
+---+----+----------+

